# RAG with FLAN-T5

In [3]:
# Packages
from RAG_Functions import *

## Embedding Model

In [8]:
# Packages
from sentence_transformers import SentenceTransformer

In [9]:
# embedding model
embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
embedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

## Chat Model

In [4]:
# Packages
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

c:\Users\ijyli\miniforge3\envs\data_engineering_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load model directly
chat_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
chat_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

## Milvus Connection

In [6]:
from pymilvus import Collection, connections
connections.connect(host='localhost', port='19530')
collection = Collection("text_embeddings")      # Get an existing collection.
# index_params = {
#     "metric_type": "COSINE",
#     "index_type": "FLAT"#,
#     #"params": {"nlist": 128}
# }
# collection.drop_index()
# collection.create_index(field_name="embedding", index_params=index_params)
# "metric_type": "L2",
#     "index_type": "IVF_FLAT",
#     "params": {"nlist": 128}
collection.load()

## Perform Chat

In [10]:
# Chat with model
input_text = input()

# Get embedding of input
input_embedding = get_mixedbread_of_query(embedding_model, input_text)

# Top5 sentences
top5_sentences = return_top_5_sentences(collection, input_embedding)

print(top5_sentences)

['["id: 449003062417443424, distance: 130.72021484375, entity: {\'company_name\': \'AppleServices\', \'sentence\': \'AppleServices: Apple also may disclose your information when Apple determines that applicable law requires or permits such disclosure, including exchanging information with other companies and organizations for fraud protection purposes.\'}", "id: 449003062417444595, distance: 130.72021484375, entity: {\'company_name\': \'AppleServices\', \'sentence\': \'AppleServices: Apple also may disclose your information when Apple determines that applicable law requires or permits such disclosure, including exchanging information with other companies and organizations for fraud protection purposes.\'}", "id: 449003062417444296, distance: 133.6761016845703, entity: {\'company_name\': \'AppleServices\', \'sentence\': \'AppleServices: Apple shares personal information with companies who provide services such as information processing, extending credit, fulfilling customer orders, deli

In [ ]:

# Tokenize
input_ids = chat_tokenizer(input_text, return_tensors="pt").input_ids

outputs = chat_model.generate(input_ids)
print(chat_tokenizer.decode(outputs[0]))